#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 3:   Collaborative Filtering + Matrix Factorization

### 100 points [6% of your final grade]

### Due: Wednesday, March 27, 2019 by 11:59pm

*Goals of this homework:* Understand how collaborative filtering and matrix factorization works. Explore different methods for real-world recommendation senarios.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw3.ipynb`. For example, my homework submission would be something like `555001234_hw3.ipynb`. Submit this notebook via eCampus (look for the homework 3 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1. Collaborative Filtering (50 points)

In this part, you will implement collaborative filtering on the [MovieLens Latest Datasets](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html). After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

As background, read [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf) and refer to the course slides `week06rec.pdf` for collaborative filtering.

## 1.1 Load the Data

Please download the dataset from Piazza. There are about 65,000 ratings in total. We split the rating data into two set. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [10]:
# load the data, then print out the number of
# movies and users in each of train and test sets.
# Your Code Here...
import pandas as pd
import numpy as np
import math

train_data = pd.read_csv('train_movie.csv')
test_data = pd.read_csv('test_movie.csv')

print('Unique users count:', len(train_data.userId.unique()+test_data.userId.unique()))
print('Unique movies count: ', len(train_data.movieId.unique()+test_data.movieId.unique()))


Unique users count: 541
Unique movies count:  1211


## 1.2 Implement User-based Collaborative Filtering

In this part, you will implement the basic User–User Collaborative Filtering algorithm introduced in the class. Given the ratings by each user, you are going to try different methods in calculating the similarities between users. You will use equation `(c)` in `week06rec.pdf` (Page 40) to aggregate ratings. Set k = 0.05. Just consider all users as neighbors. That is, while predicting how user $u$ will rate item $i$, $\widehat{C}$ includes all users who have ratings on i in the training set.

*For this memory-based algorithm, you can only rely on the ratings in training set to predict for the testing set. That is, you assume that you don't know any ratings information in the test set except that when you evalaute your model.*

### Cosine Similarity

First, you will try to calculate the similarity between users with cosine similary following the equation on page 16 of [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf). And then you need to predict the rating for each (user, movie) tuple in the test set. *Note: you don't need to subtract user mean baseline from the ratings prior to computing the similarity.*

In [0]:
# Your Code Here...
# Predict for test set
# for item in train_data.movieId.unique():
#         domain = df.index[df['movieId']==item].tolist()
def cosSim(user,target_user,target_movie,train_data,test_data):
    num = 0
    den,den1,den2,avg_rating,avg_target = 0,0,0,0,0
    #for item in train_data.userId.unique():
    domain_A = train_data.index[train_data['userId']==user].tolist()
    test_domain_A = test_data.index[test_data['userId']==user].tolist()
    domain_B = train_data.index[train_data['userId']==target_user].tolist()
    test_domain_B = test_data.index[test_data['userId']==target_user].tolist()
    mov_A,mov_B = {},{}
    for ind in domain_A:
        mov_A[train_data.at[ind,'movieId']] = train_data.at[ind,'rating']
    for ind in domain_B:
        mov_B[train_data.at[ind,'movieId']] = train_data.at[ind,'rating']
    test_mov_A,test_mov_B = {},{}
    for ind in test_domain_A:
        test_mov_A[test_data.at[ind,'movieId']] = test_data.at[ind,'rating']
    for ind in test_domain_B:
        test_mov_B[test_data.at[ind,'movieId']] = test_data.at[ind,'rating']
    comm = list(set(mov_A.keys()).intersection(mov_B.keys()))
    for movie in comm:
        num += (mov_A[movie]*mov_B[movie])
    for movie in mov_A:
        den1 += mov_A[movie]**2
    for movie in test_mov_A:
        avg_rating += test_mov_A[movie]
    for movie in mov_B:
        den2 += mov_B[movie]**2
    for movie in test_mov_B:
        avg_target += test_mov_B[movie]
    den = math.sqrt(den1)*math.sqrt(den2)
    return ((num/den), (avg_rating/len(test_domain_A)), (avg_target/len(test_domain_B)), (test_mov_A.get(target_movie,0)))


def predictModel(user_list,target_user,target_movie,train_data,test_data,k=0.05):
    r = 0
    for user in user_list:                       
        similarity, avg_user_rating, avg_target_rating, user_rating = cosSim(user,target_user,target_movie,train_data,test_data)
        r += similarity*(user_rating - avg_user_rating)
    return (avg_target_rating + k*r)

def neighbours(target_movie,target_user,train_data):
    index_list = train_data.index[train_data['movieId']==target_movie].tolist()
    user_list = [train_data.at[ind,'userId'] for ind in index_list]
    if target_user in user_list:
        user_list.remove(target_user)
    return user_list 
    
def CF(test_data,train_data):
    all_users = test_data.userId.unique()
    all_movies = test_data.movieId.unique()
    rating = {}
    for target_user in all_users[:10]:
        rating[target_user] = []
        for target_movie in all_movies[:10]:
            user_list = neighbours(target_movie,target_user,train_data)
            rating[target_user].append(predictModel(user_list,target_user,target_movie,train_data,test_data))
    return rating

predicted_rating = CF(test_data,train_data)          
       


In [30]:
for i in predicted_rating.keys():
  print(i, predicted_rating[i])
  
for i in test_data.userId.unique()[:10]:
  temp = test_data.index[test_data['userId']==i].tolist()
  for j in temp:
    print(test_data.at[j,'rating'],end=', ')
  print()
    



0 [2.3413500220332133, 2.936448600199732, 2.9763749081636464, 3.5485658934621296, 2.7060453405518166, 3.68782859802916, 3.1485475784776478, 1.176872460667565, 3.2114062835540027, 2.4906911138075003]
1 [3.8719487416640526, 4.096558154008376, 4.010431718966637, 4.140618327813155, 3.5123158839036392, 4.164689185542374, 4.022817111983288, 3.0396145400406036, 4.01641637373276, 3.6562669778102537]
2 [0.7275579713322888, 0.7799875233520351, 0.6945035907516561, 0.8802350042719611, 0.7757843737661942, 0.9522257717124902, 0.8546615578537875, 0.5998023622459202, 0.8685082531401824, 0.8467269430701704]
3 [2.018894320482487, 2.3796327325153843, 2.412131000211609, 2.78433004921818, 2.229973625645649, 2.7914383503946283, 2.4178237302586245, 1.1239821850704508, 2.466563319229225, 2.0437103136143255]
4 [2.639888130906669, 2.9186393446022576, 2.8822913701570236, 3.182812179061996, 2.8220632075875285, 3.2337789635377816, 2.9761472119406966, 2.067458938839745, 2.904243172726057, 2.5997484179463153]
5 [2.2

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [0]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test set




### Pearson correlation

Then, you will try to calculate the similarity between users with pearson correlation following `week06rec.pdf` (Page 37). And then you need to predict the rating for each (user, movie) tuple in the test set.

In [0]:
# Your Code Here...
# Predict for test set



### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [0]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



### Pearson correlation (varying the threshold)

In [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf), they observe that: 

> Pearson correlation suffers from computing high similarity
between users with few ratings in common. This can be alleviated by setting a threshold on the number of co-rated items
necessary for full agreement (correlation of 1) and scaling the
similarity when the number of co-rated items falls below this
threshold.

So now revise your Pearson to consider a threshold. Try several values and report for one that you think is appropriate.

In [0]:
# Your Code Here...
# Predict for test set



### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [0]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



What do you observe? How different are your results for the original Pearson Correlation approach vs. the thresholded version vs. the Cosine Similarity approach? Why do you think this is? *provide a brief (1-2 paragraph) discussion based on these questions.*

# Part 2. MF (20 points)

In class, we introduced how matrix factorization works for recommendation. Now it is your term to implement it. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model. *You can refer to tutorials and resources online. Remember our **collaboration policy** and you need to inform us of the resources you refer to.* 

Please report MAE and RMSE of your MF model for the test set. *You are expected to get a lower MAE and RMSE compared with the results in Part 1.*

In [0]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

# Part 3. Extension (30 points)

Given your results in the previous parts, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system, and so on. *As in the last homework, you can do anything you like to improve MAE and RMSE.*

You will get full marks for this part if you get better results than both of your CF and MF (of course we will also judge whether what you do here is reasonable or not). Additionally, you will get 5 points as bonus if your model performs the best among the whole class.

In [0]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



Please explain what you do to improve the recommendation in 1-2 paragraphs.

## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*